In [1]:
# -------------------------------------------------------- Imports
import pandas as pd
import numpy as np
import os

pd.options.display.max_rows = 1008
pd.options.display.max_columns = 100

In [2]:
# -------------------------------------------------------- Create Notebook, Path, and inital DFs
notebook_path = os.path.abspath("DataAnalysis.ipynb")
temp = os.path.abspath("Data")

# Load in all of the relevant work sheets
# We chose not to load in the worksheets that had attacks that we did not initially 
#    plan on modeling to save space and computation time

df1 = pd.read_csv(temp + "\\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv", low_memory = False)
df2 = pd.read_csv(temp + "\\Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv", low_memory = False)
df3 = pd.read_csv(temp + "\\Tuesday-WorkingHours.pcap_ISCX.csv", low_memory = False)
df4 = pd.read_csv(temp + "\\Wednesday-workingHours.pcap_ISCX.csv", low_memory = False)

In [3]:
# -------------------------------------------------------- Remove/Replace irritating values in column names
df1.columns = df1.columns.str.replace(' ', '')
df1.columns = df1.columns.str.replace('/s', '_s')

df2.columns = df2.columns.str.replace(' ', '')
df2.columns = df2.columns.str.replace('/s', '_s')

df3.columns = df3.columns.str.replace(' ', '')
df3.columns = df3.columns.str.replace('/s', '_s')

df4.columns = df4.columns.str.replace(' ', '')
df4.columns = df4.columns.str.replace('/s', '_s')

In [4]:
# -------------------------------------------------------- Combine dataframes together
full_df = df1.append(df2)
full_df = full_df.append(df3)
full_df = full_df.append(df4)

In [5]:
# -------------------------------------------------------- NaN Value Analysis
# Sums the NaN values per column
# df1.isna().sum()

# Prints the rows that are NaN values
# df1[df1.isna().any(axis=1)]


# NULL VALUES IN COLUMN "FlowBytes":
#         df1 = 1 / , df2 = 15 / 286467 , df3 = 201 / 445909 , df4 = 1008 / 692703
#         TOTAL NaN ROWS: 1228

,DestinationPort,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMax,FwdPacketLengthMin,FwdPacketLengthMean,FwdPacketLengthStd,BwdPacketLengthMax,BwdPacketLengthMin,BwdPacketLengthMean,BwdPacketLengthStd,FlowBytes_s,FlowPackets_s,FlowIATMean,FlowIATStd,FlowIATMax,FlowIATMin,FwdIATTotal,FwdIATMean,FwdIATStd,FwdIATMax,FwdIATMin,BwdIATTotal,BwdIATMean,BwdIATStd,BwdIATMax,BwdIATMin,FwdPSHFlags,BwdPSHFlags,FwdURGFlags,BwdURGFlags,FwdHeaderLength,BwdHeaderLength,FwdPackets_s,BwdPackets_s,MinPacketLength,MaxPacketLength,PacketLengthMean,PacketLengthStd,PacketLengthVariance,FINFlagCount,SYNFlagCount,RSTFlagCount,PSHFlagCount,ACKFlagCount,URGFlagCount,CWEFlagCount,ECEFlagCount,Down/UpRatio,AveragePacketSize,AvgFwdSegmentSize,AvgBwdSegmentSize,FwdHeaderLength.1,FwdAvgBytes/Bulk,FwdAvgPackets/Bulk,FwdAvgBulkRate,BwdAvgBytes/Bulk,BwdAvgPackets/Bulk,BwdAvgBulkRate,SubflowFwdPackets,SubflowFwdBytes,SubflowBwdPackets,SubflowBwdBytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,ActiveMean,ActiveStd,ActiveMax,ActiveMin,IdleMean,IdleStd,IdleMax,IdleMin,Label
6796,36812,0,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,NaN,Infinity,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,64,0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,64,0,0,0,0,0,0,2,0,0,0,7633,-1,0,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
14739,53581,0,1,1,0,0,0,0,0.0,0.0,0,0,0.0,0.0,NaN,Infinity,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,44,32,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,1,0.0,0.0,0.0,44,0,0,0,0,0,0,1,0,1,0,408,65535,0,44,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
15047,80,0,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,NaN,Infinity,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,64,0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,64,0,0,0,0,0,0,2,0,0,0,274,-1,0,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
209728,18467,0,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,NaN,Infinity,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,64,0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,64,0,0,0,0,0,0,2,0,0,0,229,-1,0,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [6]:
# -------------------------------------------------------- Label Counts

# print("DF1")
# print(df1.Label.value_counts())

# print("\nDF2")
# print(df2.Label.value_counts())

# print("\nDF3")
# print(df3.Label.value_counts())

# print("\nDF4")
# print(df4.Label.value_counts())

In [7]:
# -------------------------------------------------------- Drop attack types we are not concerened with
full_df = full_df[full_df.Label != 'Heartbleed']


#  full_df.dropna(how = 'any', subset=['DestinationPort', 'FlowDuration', 'TotalFwdPackets',
#        'TotalBackwardPackets', 'TotalLengthofFwdPackets',
#        'TotalLengthofBwdPackets', 'FwdPacketLengthMax', 'FwdPacketLengthMin',
#        'FwdPacketLengthMean', 'FwdPacketLengthStd', 'BwdPacketLengthMax',
#        'BwdPacketLengthMin', 'BwdPacketLengthMean', 'BwdPacketLengthStd',
#        'FlowBytes', 'FlowPackets', 'FlowIATMean', 'FlowIATStd', 'FlowIATMax',
#        'FlowIATMin', 'FwdIATTotal', 'FwdIATMean', 'FwdIATStd', 'FwdIATMax',
#        'FwdIATMin', 'BwdIATTotal', 'BwdIATMean', 'BwdIATStd', 'BwdIATMax',
#        'BwdIATMin', 'FwdPSHFlags', 'BwdPSHFlags', 'FwdURGFlags', 'BwdURGFlags',
#        'FwdHeaderLength', 'BwdHeaderLength', 'FwdPackets', 'BwdPackets',
#        'MinPacketLength', 'MaxPacketLength', 'PacketLengthMean',
#        'PacketLengthStd', 'PacketLengthVariance', 'FINFlagCount',
#        'SYNFlagCount', 'RSTFlagCount', 'PSHFlagCount', 'ACKFlagCount',
#        'URGFlagCount', 'CWEFlagCount', 'ECEFlagCount', 'Down/UpRatio',
#        'AveragePacketSize', 'AvgFwdSegmentSize', 'AvgBwdSegmentSize',
#        'FwdHeaderLength.1', 'FwdAvgBytes/Bulk', 'FwdAvgPackets/Bulk',
#        'FwdAvgBulkRate', 'BwdAvgBytes/Bulk', 'BwdAvgPackets/Bulk',
#        'BwdAvgBulkRate', 'SubflowFwdPackets', 'SubflowFwdBytes',
#        'SubflowBwdPackets', 'SubflowBwdBytes', 'Init_Win_bytes_forward',
#        'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
#        'ActiveMean', 'ActiveStd', 'ActiveMax', 'ActiveMin', 'IdleMean',
#        'IdleStd', 'IdleMax', 'IdleMin', 'Label'], inplace = True)

# full_df

In [8]:
full_df.Label.value_counts()

BENIGN              1097360
DoS Hulk             231073
PortScan             158930
DDoS                 128027
DoS GoldenEye         10293
FTP-Patator            7938
SSH-Patator            5897
DoS slowloris          5796
DoS Slowhttptest       5499
Name: Label, dtype: int64

In [9]:
# -------------------------------------------------------- Move 'Label' to the front of the columns list in the combined 
#                                                             packet

full_df = full_df[['Label', 'DestinationPort', 'FlowDuration', 'TotalFwdPackets',
       'TotalBackwardPackets', 'TotalLengthofFwdPackets',
       'TotalLengthofBwdPackets', 'FwdPacketLengthMax', 'FwdPacketLengthMin',
       'FwdPacketLengthMean', 'FwdPacketLengthStd', 'BwdPacketLengthMax',
       'BwdPacketLengthMin', 'BwdPacketLengthMean', 'BwdPacketLengthStd',
       'FlowBytes_s', 'FlowPackets_s', 'FlowIATMean', 'FlowIATStd',
       'FlowIATMax', 'FlowIATMin', 'FwdIATTotal', 'FwdIATMean', 'FwdIATStd',
       'FwdIATMax', 'FwdIATMin', 'BwdIATTotal', 'BwdIATMean', 'BwdIATStd',
       'BwdIATMax', 'BwdIATMin', 'FwdPSHFlags', 'BwdPSHFlags', 'FwdURGFlags',
       'BwdURGFlags', 'FwdHeaderLength', 'BwdHeaderLength', 'FwdPackets_s',
       'BwdPackets_s', 'MinPacketLength', 'MaxPacketLength',
       'PacketLengthMean', 'PacketLengthStd', 'PacketLengthVariance',
       'FINFlagCount', 'SYNFlagCount', 'RSTFlagCount', 'PSHFlagCount',
       'ACKFlagCount', 'URGFlagCount', 'CWEFlagCount', 'ECEFlagCount',
       'Down/UpRatio', 'AveragePacketSize', 'AvgFwdSegmentSize',
       'AvgBwdSegmentSize', 'FwdHeaderLength.1', 'FwdAvgBytes/Bulk',
       'FwdAvgPackets/Bulk', 'FwdAvgBulkRate', 'BwdAvgBytes/Bulk',
       'BwdAvgPackets/Bulk', 'BwdAvgBulkRate', 'SubflowFwdPackets',
       'SubflowFwdBytes', 'SubflowBwdPackets', 'SubflowBwdBytes',
       'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd',
       'min_seg_size_forward', 'ActiveMean', 'ActiveStd', 'ActiveMax',
       'ActiveMin', 'IdleMean', 'IdleStd', 'IdleMax', 'IdleMin']]

In [10]:
# ben = full_df[full_df.Label == 'BENIGN']
# dos_h = full_df[full_df.Label == 'DoS Hulk']
# p_scan = full_df[full_df.Label == 'PortScan']
# ddos = full_df[full_df.Label == 'DDoS']
# dos_ge = full_df[full_df.Label == 'DoS GoldenEye']
# ftp_pat = full_df[full_df.Label == 'FTP-Patator']
# ssh_pat = full_df[full_df.Label == 'SSH-Patator']
# dos_slowloris = full_df[full_df.Label == 'DoS slowloris']
# dos_slowhttp = full_df[full_df.Label == 'DoS Slowhttptest']

In [11]:
# olist = list(ben.columns) # original list
# yvlist = list(['Label']) # y variable list

# # make a new list of only the values I want to compare
# xlist = [x for x in olist if x not in yvlist]

# fig = plt.figure(figsize=(20, 25))
# plotNum = 1
# for key in xlist:
#     ax = plt.subplot(6, 3, plotNum)
#     ax.scatter(x=ben[key], y=ben['Label'])
#     ax.set_xlabel(key)
#     ax.set_ylabel('Label')
#     plotNum = plotNum + 1

In [12]:
# # Making a correlation map
# corr = ben.corr()
# plt.figure(figsize=(79, 79))
# heatMap = sns.heatmap(
#     corr,
#     vmin = -1, vmax = 1, center = 0,
#     annot=True,
#     cmap=sns.diverging_palette(20, 220, n=200),
#     square=True
# )

In [13]:
# ben.dtypes